In [261]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [141]:
data = pd.read_csv('/content/drive/MyDrive/Invest Machine/Data/tmp.csv')

In [171]:
data.head()

,timestamp,exchange,symbol,timestamp.1,local_timestamp,asks[0].price,asks[1].price,asks[2].price,asks[3].price,asks[4].price,asks[5].price,asks[6].price,asks[7].price,asks[8].price,asks[9].price,asks[0].amount,asks[1].amount,asks[2].amount,asks[3].amount,asks[4].amount,asks[5].amount,asks[6].amount,asks[7].amount,asks[8].amount,asks[9].amount,bids[0].price,bids[1].price,bids[2].price,bids[3].price,bids[4].price,bids[5].price,bids[6].price,bids[7].price,bids[8].price,bids[9].price,bids[0].amount,bids[1].amount,bids[2].amount,bids[3].amount,bids[4].amount,bids[5].amount,bids[6].amount,bids[7].amount,bids[8].amount,bids[9].amount
0,2021-04-02 09:00:00.400,deribit,BTC-16APR21,1617354000424000,1617354000430645,60892.5,60893.0,60897.0,60897.5,60904.0,60913.5,60918.5,60938.5,60942.0,60948.5,2000.0,13700.0,4180.0,6000.0,193180.0,100000.0,2540.0,8000.0,1810.0,289770.0,60828.0,60806.5,60785.0,60780.5,60780.0,60761.0,60738.5,60732.0,60725.0,60705.5,213250.0,2820.0,1810.0,2690.0,319870.0,5270.0,2290.0,379100.0,100000.0,1830.0
1,2021-04-02 09:00:00.500,deribit,BTC-16APR21,1617354000595000,1617354000606282,60892.5,60893.0,60897.5,60904.0,60913.5,60918.5,60938.5,60942.0,60948.5,60953.5,6180.0,13700.0,6000.0,193180.0,100000.0,2540.0,8000.0,1810.0,289770.0,2740.0,60828.0,60806.5,60785.0,60780.5,60780.0,60761.0,60738.5,60732.0,60725.0,60705.5,213250.0,2820.0,1810.0,2690.0,319870.0,5270.0,2290.0,379100.0,100000.0,1830.0
2,2021-04-02 09:00:00.700,deribit,BTC-16APR21,1617354000788000,1617354000794703,60892.5,60893.0,60897.5,60904.0,60913.5,60918.5,60938.5,60942.0,60948.5,60953.5,2000.0,13700.0,6000.0,193180.0,100000.0,2540.0,8000.0,1810.0,289770.0,2740.0,60828.0,60806.5,60785.0,60780.5,60780.0,60761.0,60738.5,60732.0,60725.0,60705.5,213250.0,2820.0,1810.0,2690.0,319870.0,5270.0,2290.0,379100.0,100000.0,1830.0
3,2021-04-02 09:00:00.800,deribit,BTC-16APR21,1617354000892000,1617354000905202,60892.5,60893.0,60897.5,60904.0,60913.5,60918.5,60938.5,60942.0,60948.5,60953.5,2000.0,13700.0,6000.0,193180.0,100000.0,2540.0,8000.0,1810.0,289770.0,2740.0,60828.0,60806.5,60785.0,60780.5,60780.0,60761.0,60738.5,60732.0,60725.0,60705.5,213250.0,2820.0,1810.0,2690.0,319870.0,5270.0,2290.0,379100.0,100000.0,1830.0
4,2021-04-02 09:00:00.900,deribit,BTC-16APR21,1617354000917000,1617354000926413,60892.5,60897.5,60904.0,60913.5,60918.5,60938.5,60942.0,60948.5,60953.5,60993.5,2000.0,6000.0,193180.0,100000.0,2540.0,8000.0,1810.0,289770.0,2740.0,343430.0,60828.0,60806.5,60785.0,60780.5,60780.0,60761.0,60738.5,60732.0,60725.0,60705.5,213250.0,2820.0,1810.0,2690.0,319870.0,5270.0,2290.0,379100.0,100000.0,1830.0


In [302]:
def simulation(data, ask_size):
  order_book = pd.DataFrame(columns = ['timestamp', 'amount', 'price', 'side', 'slippage'])

  NextTimeToTrade = datetime.strptime(data.iloc[0:1, 0].values[0], '%Y-%m-%d %H:%M:%S.%f')

  for i in range(len(data)):
    curr_time = datetime.strptime(data.iloc[i : i + 1, 0].values[0], '%Y-%m-%d %H:%M:%S.%f')

    if curr_time >= NextTimeToTrade:
      NextTimeToTrade += timedelta(minutes=1)

      curr_bucket = pd.DataFrame(data.iloc[i : i + 1, :])
      curr_amount = ask_size
      pinned_price = curr_bucket['asks[9].price'].values[0]


      for j in range(bucket_size):
        side = None
        if j == 0:
          side = 'taker'
        else:
          side = 'maker'

        aval_price = curr_bucket['asks[' + str(j) + '].price'].values[0] 
        aval_amount = curr_bucket['asks[' + str(j) + '].amount'].values[0]

        if aval_amount >= curr_amount:
          new_order = {'timestamp' : curr_time.strftime('%Y-%m-%d %H:%M:%S.%f'), 'amount':curr_amount, 'price':aval_price, 'side':side, 'slippage': pinned_price - aval_price}
                       
          order_book = order_book.append(new_order, ignore_index=True)
          curr_amount = ask_size
          break

        else:
          new_order = {'timestamp' : curr_time.strftime('%Y-%m-%d %H:%M:%S.%f'), 'amount':aval_amount, 'price':aval_price, 'side':side, 'slippage': pinned_price - aval_price}

          order_book = order_book.append(new_order, ignore_index=True)
          curr_amount -= aval_amount
          
  return order_book

In [303]:
def strategy_parameters(order_book, maker, taker):
  amount = 0
  total_price = 0
  total_slippage = 0

  for i in range(len(order_book)):
    curr_order = pd.DataFrame(order_book.iloc[i : i + 1, :])
    if curr_order['side'].values[0] == 'maker':
      commission = 1 + maker
    elif curr_order['side'].values[0] == 'taker':
      commission = 1 + taker
  
    total_price += curr_order['amount'].values[0] * curr_order['price'].values[0] * commission
    total_slippage += curr_order['slippage'].values[0] * curr_order['amount'].values[0]
    amount += curr_order['amount'].values[0]

  return amount, total_price / amount, total_slippage / amount

In [304]:
maker = 0.02 * 0.01
taker = 0.07 * 0.01
ask_sizes = [1000, 10000, 1000000]
results = pd.DataFrame(columns = ['minute_amount', 'total_amount', 'avg_price', 'avg_slippage'])

for ask_size in ask_sizes:

  order_book = simulation(data, ask_size)

  total_amount, avg_price, avg_slippage = strategy_parameters(order_book, maker, taker)
  new_row = {'minute_amount' : ask_size, 'total_amount':total_amount, 'avg_price':avg_price, 'avg_slippage':avg_slippage}
  results = results.append(new_row, ignore_index=True)
  
  order_book = order_book[['timestamp', 'amount', 'price', 'side']]
  order_book.to_csv('order_book_' + str(ask_size) + '.csv', index=True)

In [305]:
results

,minute_amount,total_amount,avg_price,avg_slippage
0,1000.0,61000.0,60922.844341,86.744508
1,10000.0,610000.0,60911.756584,83.606648
2,1000000.0,41828640.0,60955.226539,41.827541
